# Tratativa e merge dos dados

Neste arquivo será realizado o tratamento e o merge dos dados da operadora de saúde:

- Importação dos dados
- Análise da estrutura e estatística dos dados
- Merge dos dados
- Missings
- Type dos dados
- Criação de colunas (id_genero, imc, class_imc)
- Ordenação das colunas

### Importando bibliotecas e verificando as versões

Importar todas as bibliotecas que serão utilizadas para o tratamento dos dados e verificar as versões para controle

In [1]:
# Importando as bibliotecas que serão utilizadas para a manipulação dos dataframes

import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
# Verificação das bibliotecas usadas

print(f'''Verificando as versões das bibliotecas:\n pandas:{pd.__version__}''')

Verificando as versões das bibliotecas:
 pandas:1.5.3


### Importação de dados

Importando todos os dataframes para realizar a análise, tratamento e merge.

In [3]:
# Estados

estados = pd.read_csv('..\..\data\estados_brasileiros.csv', sep= ';', encoding= 'latin-1')

In [4]:
# Idade dos Clientes

idade = pd.read_csv('..\..\data\idade_clientes.csv', sep= ';')

In [5]:
# Clientes da Operadora de Saúde

clientes = pd.read_excel('..\..\data\clientes_operadora_saude.xlsx')

### Merge datasets

Mesclando os dataframes

In [55]:
# Realizando merge nas tabelas, unificando-as

op_saude = pd.merge(clientes, estados, right_on = 'cod_estado', left_on = 'id_estado')
op_saude = pd.merge(op_saude, idade, on= 'id_cliente').sort_values('id_cliente').reset_index(drop= True)

### Analisando os dados

Realizando a análise de todos os dados coletados para tratá-los

In [56]:
# Estrutura dos dados

op_saude.head()

,id_cliente,altura,peso,colesterol,genero,id_estado,cod_estado,estado,sigla_estado,pais,idade
0,1,1.87,102.0,190,Masculino,23,23,Roraima,RR,Brasil,17.0
1,2,1.82,96.0,170,Masculino,7,7,Distrito Federal,DF,Brasil,28.0
2,3,1.90,115.0,216,Masculino,4,4,Amazonas,AM,Brasil,62.0
3,4,1.90,78.0,146,Feminino,24,24,Santa Catarina,SC,Brasil,55.0
4,5,1.71,130.0,243,Masculino,26,26,Sergipe,SE,NaN,44.0


In [57]:
# Estatística dos dados

op_saude.describe()

,id_cliente,altura,peso,colesterol,id_estado,cod_estado,idade
count,225.000000,222.000000,222.000000,225.000000,225.000000,225.000000,223.000000
mean,113.000000,1.746171,89.265766,172.591111,13.711111,13.711111,42.865471
std,65.096083,0.105304,21.395153,40.072723,7.562307,7.562307,15.292046
min,1.000000,1.500000,50.000000,96.000000,1.000000,1.000000,16.000000
25%,57.000000,1.680000,72.000000,139.000000,7.000000,7.000000,30.000000
50%,113.000000,1.750000,90.500000,172.000000,14.000000,14.000000,43.000000
75%,169.000000,1.820000,103.750000,209.000000,20.000000,20.000000,55.000000
max,225.000000,1.980000,138.000000,256.000000,27.000000,27.000000,70.000000


In [58]:
# Soma dos dados missing

op_saude.isna().sum()

id_cliente       0
altura           3
peso             3
colesterol       0
genero           0
id_estado        0
cod_estado       0
estado           0
sigla_estado     0
pais            25
idade            2
dtype: int64

In [59]:
# Informações dos dados

op_saude.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_cliente    225 non-null    int64  
 1   altura        222 non-null    float64
 2   peso          222 non-null    float64
 3   colesterol    225 non-null    int64  
 4   genero        225 non-null    object 
 5   id_estado     225 non-null    int64  
 6   cod_estado    225 non-null    int64  
 7   estado        225 non-null    object 
 8   sigla_estado  225 non-null    object 
 9   pais          200 non-null    object 
 10  idade         223 non-null    float64
dtypes: float64(3), int64(4), object(4)
memory usage: 19.5+ KB


In [60]:
# Tamanho da base de dados

op_saude.shape

(225, 11)

### Tratamento dos dados

Tratando missings com drop e substituição por moda e média por genero

##### Missings


In [22]:
# Verificando todos os dados ausentes

op_saude.isnull().sum()

id_cliente       0
altura           3
peso             3
colesterol       0
genero           0
id_estado        0
cod_estado       0
estado           0
sigla_estado     0
pais            25
idade            2
dtype: int64

##### Missing - Eliminando dados


In [61]:
# Dados ausentes do df clientes

op_saude.loc[op_saude['idade'].isnull()] # os dados ausentes do peso e altura estão nas mesmas linhas 

,id_cliente,altura,peso,colesterol,genero,id_estado,cod_estado,estado,sigla_estado,pais,idade
20,21,1.79,83.0,189,Feminino,5,5,Bahia,BA,Brasil,NaN
48,49,1.73,67.0,154,Feminino,7,7,Distrito Federal,DF,Brasil,NaN


In [62]:
# Eliminando os dados

op_saude.dropna(subset=['idade'], inplace = True)

In [63]:
# Verificando os dados "idade"

op_saude.idade.isnull().sum()

0

##### Missing - Substituição (média/mediana)

In [64]:
# Descobrindo a média e mediana do peso por genero feminino e masculino

op_saude.groupby('genero').peso.describe()

,count,mean,std,min,25%,50%,75%,max
genero,,,,,,,,
Feminino,108.0,76.935185,16.064865,50.0,62.00,76.0,91.0,117.0
Masculino,112.0,101.410714,19.000500,54.0,88.75,101.5,118.0,138.0


In [65]:
# Descobrindo a média e mediana da altura por genero feminino e masculino

op_saude.groupby('genero').altura.describe()

,count,mean,std,min,25%,50%,75%,max
genero,,,,,,,,
Feminino,108.0,1.697963,0.094846,1.50,1.6200,1.69,1.760,1.91
Masculino,112.0,1.792411,0.094700,1.57,1.7275,1.80,1.845,1.98


In [66]:
# Como a média e mediana do peso e da altura são muito próximas, cria-se df com média para substituição

med_peso = op_saude.groupby('genero')['peso'].transform('mean')
med_altura = op_saude.groupby('genero')['altura'].transform('mean')

In [67]:
# Localizando os dados com o missing no peso

op_saude.loc[op_saude['peso'].isnull()]

,id_cliente,altura,peso,colesterol,genero,id_estado,cod_estado,estado,sigla_estado,pais,idade
32,33,NaN,NaN,188,Masculino,22,22,Rondonia,RO,Brasil,56.0
41,42,NaN,NaN,177,Masculino,20,20,Rio Grande do Norte,RN,NaN,54.0
50,51,NaN,NaN,138,Feminino,26,26,Sergipe,SE,NaN,23.0


In [68]:
# Alterando o peso e altura pela média por genero

op_saude['peso'].fillna(med_peso, inplace=True)
op_saude['altura'].fillna(med_altura, inplace=True)

In [69]:
# Localizando os dados que eram missing, para verificar os valores alterados

op_saude.loc[[32, 41, 50]]  

,id_cliente,altura,peso,colesterol,genero,id_estado,cod_estado,estado,sigla_estado,pais,idade
32,33,1.792411,101.410714,188,Masculino,22,22,Rondonia,RO,Brasil,56.0
41,42,1.792411,101.410714,177,Masculino,20,20,Rio Grande do Norte,RN,NaN,54.0
50,51,1.697963,76.935185,138,Feminino,26,26,Sergipe,SE,NaN,23.0


##### Missing - Substituição (moda)

In [70]:
# Verificando os dados ausentes categóricos

op_saude.pais.unique()

array(['Brasil', nan], dtype=object)

In [71]:
# Criando variável com a ccorreção dos termos em "país"

correcao_pais = op_saude.pais.mode()[0]

In [72]:
# Substituição dos missings

op_saude.pais.fillna(correcao_pais, inplace= True)

In [73]:
# Verificando os dados da coluna "país"

op_saude.pais.isna().sum()

0

##### Verificando missings

In [74]:
# Verificando se todos os missings foram tratados

op_saude.isnull().sum()

id_cliente      0
altura          0
peso            0
colesterol      0
genero          0
id_estado       0
cod_estado      0
estado          0
sigla_estado    0
pais            0
idade           0
dtype: int64

### Organizando os dados

Organização e criação de colunas para enriquecer os dados para a análise exploratória

##### Transformando idade e peso em números inteiros

In [75]:
# Alterando o type das colunas 'idade' e 'peso'

op_saude['idade'] = op_saude['idade'].astype(int)
op_saude['peso'] = op_saude['peso'].astype(int)

##### Criando função para categorizar o genero

Para facilitar a utilização dos dados 'genero', estou criando coluna categorizando-os em 0 e 1, o que facilita a utilização desses dados em aprendizado de máquina e plotagem de gráficos

In [76]:
# Atribuindo função a uma variável

LE = LabelEncoder()

In [77]:
# Criando uma variável categórica, chamada id_genero, na transformação do LE - Label Enconder de clientes['genero']

op_saude['id_genero'] = LE.fit_transform(op_saude['genero'])

In [78]:
# Visualização dos dados

op_saude.head()

,id_cliente,altura,peso,colesterol,genero,id_estado,cod_estado,estado,sigla_estado,pais,idade,id_genero
0,1,1.87,102,190,Masculino,23,23,Roraima,RR,Brasil,17,1
1,2,1.82,96,170,Masculino,7,7,Distrito Federal,DF,Brasil,28,1
2,3,1.90,115,216,Masculino,4,4,Amazonas,AM,Brasil,62,1
3,4,1.90,78,146,Feminino,24,24,Santa Catarina,SC,Brasil,55,0
4,5,1.71,130,243,Masculino,26,26,Sergipe,SE,Brasil,44,1


##### Criando coluna IMC

Como no dataframe possui a coluna de altura e peso, é possível criar uma coluna com o imc e com ela se obtem o grau do peso de cada paciente. O IMC vai servir como parametro para análise das causas do colesterol.

In [79]:
# Criando coluna com o IMC de cada um dos clientes da operadora de saúde

op_saude['imc'] = round(op_saude.peso / (op_saude.altura * op_saude.altura), 2)
op_saude.head()

,id_cliente,altura,peso,colesterol,genero,id_estado,cod_estado,estado,sigla_estado,pais,idade,id_genero,imc
0,1,1.87,102,190,Masculino,23,23,Roraima,RR,Brasil,17,1,29.17
1,2,1.82,96,170,Masculino,7,7,Distrito Federal,DF,Brasil,28,1,28.98
2,3,1.90,115,216,Masculino,4,4,Amazonas,AM,Brasil,62,1,31.86
3,4,1.90,78,146,Feminino,24,24,Santa Catarina,SC,Brasil,55,0,21.61
4,5,1.71,130,243,Masculino,26,26,Sergipe,SE,Brasil,44,1,44.46


##### Criando coluna de classificação do IMC

Para facilitar a vizualização da classificação do IMC, cria-se uma coluna com essa informação.

In [80]:
#               IMC - tabela
#    __________________________________
#        Baixo Peso    |     < 18        
#       Normal Peso    |   18 - 24.9        
#        Sobrepeso     |   25 - 29.9
#     Obesidade Grau 1 |   30 - 34.9
#     Obesidade Grau 2 |   35 - 39.9
#     Obesidade Grau 3 |     > 40

In [81]:
# Tabela da classificação de peso de acordo com o valor do IMC de cada paciente

tabela_imc = [[0, 18, 'Baixo Peso'],
              [18, 25, 'Normal Peso'],
              [25, 30, 'Sobrepeso'],
              [30, 35, 'Obesidade Grau 1'],
              [35, 40, 'Obesidade Grau 2'],
              [40, 100, 'Obesidade Grau 3']]

In [82]:
# Estrutura de repetição para criar a classificação no dataframe

for imc in tabela_imc:
    op_saude.loc[(op_saude['imc'] > imc[0]) & (op_saude['imc'] < imc[1]), 'class_imc'] = imc[2]

In [83]:
# Visualização do dataframe

op_saude.head()

,id_cliente,altura,peso,colesterol,genero,id_estado,cod_estado,estado,sigla_estado,pais,idade,id_genero,imc,class_imc
0,1,1.87,102,190,Masculino,23,23,Roraima,RR,Brasil,17,1,29.17,Sobrepeso
1,2,1.82,96,170,Masculino,7,7,Distrito Federal,DF,Brasil,28,1,28.98,Sobrepeso
2,3,1.90,115,216,Masculino,4,4,Amazonas,AM,Brasil,62,1,31.86,Obesidade Grau 1
3,4,1.90,78,146,Feminino,24,24,Santa Catarina,SC,Brasil,55,0,21.61,Normal Peso
4,5,1.71,130,243,Masculino,26,26,Sergipe,SE,Brasil,44,1,44.46,Obesidade Grau 3


##### Criando coluna de classificação do colesterol

Para facilitar a vizualização da classificação do colesterol, cria-se uma coluna com essa informação

In [84]:
#           Colesterol - tabela
#    __________________________________
#       Muito Baixo   |      < 50        
#         Moderado    |   50 - 199.9        
#           Alto      |   200 - 239.9
#        Muito Alto   |      > 240


In [85]:
# Tabela da classificação do colesterol de acordo com o valor do colesterol de cada paciente

tabela_colesterol = [[0, 50, 'Muito baixo'], 
                     [50, 200, 'Moderado'],
                     [200, 240, 'Alto'],
                     [240, 600, 'Muito Alto']]


In [86]:
# Estrutura de repetição para criar a classificação no dataframe

for colest in tabela_colesterol:
    op_saude.loc[(op_saude['colesterol'] > colest[0]) & (op_saude['imc'] < colest[1]), 'class_colest'] = colest[2]

##### Dropando coluna repetida

In [87]:
# Removendo a coluna 'cod_estado'

op_saude.drop(columns= 'cod_estado', inplace= True)

##### Alterando conteúdo da variável siglas do estado

In [88]:
# Padronizando as siglas dos estados

op_saude['sigla_estado'] = op_saude.sigla_estado.str.upper()

#### Alterando disposições das colunas

In [89]:
# Alterando a ordem das colunas do dataframe

ordem_colunas = ['id_cliente', 'id_genero', 'genero', 'idade', 'peso', 'altura', 'imc', 'class_imc', 'colesterol', 'class_colest', 'estado', 'sigla_estado', 'pais']
op_saude[ordem_colunas].head()

,id_cliente,id_genero,genero,idade,peso,altura,imc,class_imc,colesterol,class_colest,estado,sigla_estado,pais
0,1,1,Masculino,17,102,1.87,29.17,Sobrepeso,190,Moderado,Roraima,RR,Brasil
1,2,1,Masculino,28,96,1.82,28.98,Sobrepeso,170,Moderado,Distrito Federal,DF,Brasil
2,3,1,Masculino,62,115,1.90,31.86,Obesidade Grau 1,216,Alto,Amazonas,AM,Brasil
3,4,0,Feminino,55,78,1.90,21.61,Normal Peso,146,Moderado,Santa Catarina,SC,Brasil
4,5,1,Masculino,44,130,1.71,44.46,Obesidade Grau 3,243,Muito Alto,Sergipe,SE,Brasil


### Arquivo CSV para a EDA

In [90]:
# Criando arquivo, na pasta tratativa, do dataframe 'op_saude' com todo o tratamento realizado neste arquivo

op_saude.to_csv('merged_operadora_saude.csv')